In [1]:
import os
import random
from tifffile import imread, imwrite

def clear_directory(directory):
    """Delete all files in the specified directory."""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            os.rmdir(file_path)

def split_tiff(tiff_path, train_dir, test_dir, indices, is_label=False):
    # Read the TIFF file
    tiff_data = imread(tiff_path)

    # Check if the TIFF file contains multiple slices
    if len(tiff_data.shape) != 3:
        print("The TIFF file does not contain multiple slices.")
        return

    # Create the output directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Clear the output directories
    clear_directory(train_dir)
    clear_directory(test_dir)

    # Initialize counters for train and test slices
    train_counter = 0
    test_counter = 0

    # Save the slices into the appropriate directories
    for i, slice_data in enumerate(tiff_data):
        if i in indices['test']:
            if is_label:
                slice_filename = f"image_{test_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{test_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(test_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Test)")
            test_counter += 1
        else:
            if is_label:
                slice_filename = f"image_{train_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{train_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(train_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Train)")
            train_counter += 1

        imwrite(slice_path, slice_data)

    print("Slicing complete.")

def determine_split_indices(total_slices, test_ratio=0.2):
    indices = list(range(total_slices))
    random.shuffle(indices)
    split_point = int(total_slices * test_ratio)
    return {'test': indices[:split_point], 'train': indices[split_point:]}

# Specify the path to your image and label TIFF files
image_tiff_path = r"/argusdata/users/undergrads/ItamarAndDafna/Final_Project_rawdata/10_30_HIGH_5_LOW 2CR/M1/Interpolated_GlyR_M1_stack_dafna.tif"
label_tiff_path = r"/argusdata/users/undergrads/ItamarAndDafna/Final_Project_rawdata/10_30_HIGH_5_LOW 2CR/M1/initial_mask_substack.tif"

# Specify the output directories for training and testing
image_train_dir = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr"
image_test_dir = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTs"
label_train_dir = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/labelsTr"
label_test_dir = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/labelsTs"  # Ground truth information for comparison later

# Read the image TIFF file to determine the number of slices
tiff_data = imread(image_tiff_path)
total_slices = tiff_data.shape[0]

# Determine the split indices
indices = determine_split_indices(total_slices, test_ratio=0.2)

# Call the function to split the image TIFF file into individual slices (with channel suffix)
split_tiff(image_tiff_path, image_train_dir, image_test_dir, indices)

# Call the function to split the label TIFF file into individual slices (without channel suffix)
split_tiff(label_tiff_path, label_train_dir, label_test_dir, indices, is_label=True)


Slice 1 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_001_0000.tif (Train)
Slice 2 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_002_0000.tif (Train)
Slice 3 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_003_0000.tif (Train)
Slice 4 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_004_0000.tif (Train)
Slice 5 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_005_0000.tif (Train)
Slice 6 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_006_0000.tif (Train)
Slice 7 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/image_007_0000.tif (Train)
Slice 8 saved as /argusdata/users/undergrads/Ita

# The following code is:
### 1. Slicing the image
### 2. creating labelsTs and imagesTs folders that include all the slices
### 3. shuffling and splitting the slices to train and test sets and creating a csv file with a record of which slices assigned to which set
### 4. creating folder of labelsTr, imagesTr, labelsTs, imagesTs

### keep notice of the paths! and  manually open folder of {dataset_name} and {dataset_name}_noSplit in the /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw folder

In [3]:
import os
import random
import csv
from tifffile import imread, imwrite

def clear_directory(directory):
    """Delete all files in the specified directory."""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            os.rmdir(file_path)

def save_all_slices(tiff_data, all_slices_dir, is_label=False):
    """Save all slices from a TIFF file to the specified directory."""
    os.makedirs(all_slices_dir, exist_ok=True)
    clear_directory(all_slices_dir)
    
    for i, slice_data in enumerate(tiff_data):
        if is_label:
            slice_filename = f"image_{i+1:03d}.tif"  # No channel suffix for labels
        else:
            slice_filename = f"image_{i+1:03d}_0000.tif"  # Channel suffix for images
        slice_path = os.path.join(all_slices_dir, slice_filename)
        imwrite(slice_path, slice_data)

    print(f"All slices saved in {all_slices_dir}.")

def split_tiff(tiff_path, train_dir, test_dir, indices, csv_writer, is_label=False):
    """Split a TIFF file into training and testing slices and save them."""
    # Read the TIFF file
    tiff_data = imread(tiff_path)

    # Check if the TIFF file contains multiple slices
    if len(tiff_data.shape) != 3:
        print("The TIFF file does not contain multiple slices.")
        return

    # Create the output directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Clear the output directories
    clear_directory(train_dir)
    clear_directory(test_dir)

    # Initialize counters for train and test slices
    train_counter = 0
    test_counter = 0

    # Save the slices into the appropriate directories
    for i, slice_data in enumerate(tiff_data):
        if i in indices['test']:
            if is_label:
                slice_filename = f"image_{test_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{test_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(test_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Test)")
            test_counter += 1
            csv_writer.writerow([i+1, 'Test', slice_path])
        else:
            if is_label:
                slice_filename = f"image_{train_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{train_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(train_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Train)")
            train_counter += 1
            csv_writer.writerow([i+1, 'Train', slice_path])

        imwrite(slice_path, slice_data)

    print("Slicing complete.")

def determine_split_indices(total_slices, test_ratio=0.2):
    """Determine indices for splitting slices into train and test sets."""
    indices = list(range(total_slices))
    random.shuffle(indices)
    split_point = int(total_slices * test_ratio)
    return {'test': indices[:split_point], 'train': indices[split_point:]}

# Specify the path to your image and label TIFF files
image_tiff_path = r"/argusdata/users/undergrads/ItamarAndDafna/Final_Project_rawdata/12_30_HIGH_5_LOW 3CM/M1 done/Interpolated GlyR M1 stack.tif"
label_tiff_path = r"/argusdata/users/undergrads/ItamarAndDafna/Final_Project_rawdata/12_30_HIGH_5_LOW 3CM/M1 done/initial mask.tiff"

dataset_name = 'Dataset032_1230H5L_3CM_M1'

# Specify the output directories for training and testing
image_train_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}/imagesTr"
image_test_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}/imagesTs"
label_train_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}/labelsTr"
label_test_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}/labelsTs"  # Ground truth information for comparison later

# Specify the directories for saving all slices
all_slices_image_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}_noSplit/imagesTs"
all_slices_label_dir = f"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/{dataset_name}_noSplit/labelsTs"

# Read the image TIFF file to determine the number of slices
tiff_data = imread(image_tiff_path)
total_slices = tiff_data.shape[0]

# Determine the split indices
indices = determine_split_indices(total_slices, test_ratio=0.2)

# Save all slices before splitting
save_all_slices(imread(image_tiff_path), all_slices_image_dir)
save_all_slices(imread(label_tiff_path), all_slices_label_dir, is_label=True)

# Open a CSV file to write the records
csv_file_path = f"/argusdata/users/undergrads/ItamarAndDafna/slicing_info/split_records_{dataset_name}.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Slice Number', 'Set', 'Path'])

    # Call the function to split the image TIFF file into individual slices (with channel suffix)
    split_tiff(image_tiff_path, image_train_dir, image_test_dir, indices, csv_writer)

    # Call the function to split the label TIFF file into individual slices (without channel suffix)
    split_tiff(label_tiff_path, label_train_dir, label_test_dir, indices, csv_writer, is_label=True)

print(f"Records saved to {csv_file_path}")


All slices saved in /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1_noSplit/imagesTs.
All slices saved in /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1_noSplit/labelsTs.
Slice 1 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1/imagesTr/image_001_0000.tif (Train)
Slice 2 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1/imagesTr/image_002_0000.tif (Train)
Slice 3 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1/imagesTr/image_003_0000.tif (Train)
Slice 4 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1/imagesTr/image_004_0000.tif (Train)
Slice 5 saved as /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset032_1230H5L_3CM_M1/imagesTr/image_005_0000.tif (Train)
Slice 6 saved as /argusdata/users/undergrads/Itama